# Ensembles and Voting

by Casey Durfee <hello@caseydurfee.com>

AI and machine learning is often used for classification. Is this mushroom safe to eat? Is this person a good or bad credit risk? Is this a cat or a dog?

Different machine learning algorithms have different strengths and weaknesses.

So it's natural to think that combining multiple models together will produce better (or at least more robust) results than the individual classifiers. This is called an ensemble. there are techniques like gradient boosting and random forests that are ensembles of very simple models (weak learners). We can also make ensembles of more powerful systems (strong learners). The most common way to do that is with voting. Query several classifiers, and have the ensemble return the majority pick.

I've created an extremely simple model of an ensemble. I generate a random sequence of booleans as the ground truth, then create a bunch of fake classifiers by flipping 40% of the bits. Although simplistic, this model shows a few things to know about using ensembles in the real world.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode
from sklearn.metrics import f1_score, classification_report

rng = np.random.default_rng(2718)

def create_ground_truth(num_bits):
    return [rng.random() > .5 for x in range(num_bits)]

In [2]:
NUM_BITS = 100000

ground_truth = create_ground_truth(NUM_BITS)

ground_truth[:10]

[False, False, True, True, False, False, False, False, False, False]

In [ ]:
def create_classifiers(ground_truth, num_classifiers, flip_ratio=.4):
    """
    emulate a bunch of classifiers that we're using together in an ensemble.
    
    flips a certain fraction of bits, according to `flip_ratio`
    """
    classifiers = [ground_truth.copy() for x in range(num_classifiers)]

    for classifier in classifiers:
        for count, bit in enumerate(classifier):
            if rng.random() < flip_ratio:
                classifier[count] = not(bit)
    return classifiers


Each of the systems agree with the ground truth about 60% of the time, as expected.

In [4]:
classifiers = create_classifiers(ground_truth, 999)

print(f1_score(ground_truth, classifiers[0]))
print(f1_score(ground_truth, classifiers[1]))
print(f1_score(ground_truth, classifiers[2]))

0.5982283897542541
0.6033461622828883
0.5991949050708965


They agree with each other pairwise about 52% of the time.

In [5]:
sum(pd.Series(classifiers[0]) == pd.Series(classifiers[1])) / len(classifiers[1])

0.5194

In [6]:
sum(pd.Series(classifiers[2]) == pd.Series(classifiers[1])) / len(classifiers[1])

0.52112

In [7]:
sum(pd.Series(classifiers[0]) == pd.Series(classifiers[2])) / len(classifiers[2])

0.51972

This is as uncorrelated as they can be given that each one is correct 60% of the time, and wrong 40% of the time.

If two of these fake classifiers agree, either they're both right, or they're both wrong.

If they're totally independent of each other, they will both be right `60% * 60% = 36%` of the time, and both wrong `40% * 40% = 16%`. So they will agree `36% + 16% = 52%` of the time at minimum.

In [8]:
## this will only work for binary classification. if multiclass, I'd use the mode() function

def do_majority(classifiers):
    """
    performant approach for binary classification. take the mean value. if mean is > .5 then majority of classifiers voted 1
    """
    return np.round(np.mean(classifiers, axis=0))

With 3 classifiers, the ensemble f1 score (0.65) is higher than the individual classifiers (.6).

In [27]:
majority2 = do_majority(classifiers[0:3])
print(classification_report(ground_truth, majority2, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65     49918
           1       0.65      0.64      0.65     50082

    accuracy                           0.65    100000
   macro avg       0.65      0.65      0.65    100000
weighted avg       0.65      0.65      0.65    100000



The f1 score will continue to go up with the more classifiers added. With 999 classifiers, the ensemble gets a perfect score.

In [10]:

print(classification_report(ground_truth, do_majority(classifiers), target_names=['0', '1']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49918
           1       1.00      1.00      1.00     50082

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000



## Theory vs reality

In the real world, there are diminishing returns to adding more classifiers to the ensemble. On paper, there aren't.

What is the probability of the ensemble being wrong about a particular classification?

That's the probability that the majority of the classifiers predict 0, given that the true value is 1. If each classifier is more likely to be right than wrong, as the number of classifiers goes to infinity, the probability of the majority of predictions being wrong goes to 0.

Say there are 1,000 components, each of which has a 60% chance of being right. So we'd expect 600 of them to give the right answer. For the ensemble to give the wrong answer, it would need to be 500 or fewer. We could calculate that exact probability using the binomial distribution, or the normal approximation of it.

If each binary classifier has a probability > .5 of being right, we can make the ensemble arbitrarily precise if we add enough of them to the ensemble (assuming their errors are independent).

Let's say each classifier is only right 50.5% of the time. We might have to add 100,000 of them to the ensemble, but we can make the error rate arbitrarily small.

In [11]:
import scipy.stats
NUM_CLASSIFIERS = 100000
PROB_OF_BEING_RIGHT = .505

components = scipy.stats.binom(NUM_CLASSIFIERS, PROB_OF_BEING_RIGHT)

comp_df = pd.DataFrame({
    'x': [x for x in range(NUM_CLASSIFIERS+1)],
    'y': [components.pmf(x) for x in range(NUM_CLASSIFIERS+1)]
})

ensemble_error_rate = sum(comp_df[comp_df.x < (NUM_CLASSIFIERS/2)].y)
print(ensemble_error_rate)

0.000774011660697044



In this simple model, there's no point where adding more classifiers to an ensemble stops making it more accurate, or starts making it worse. 

In the real world, I've found that there's a hard limit to how good ensembles can be.

## Correlated models ruin ensembles

The big difference between my experiment and reality is that the errors each classifier makes are uncorrelated with each other. The more each classifier's wrong answers are correlated with other classifiers' wrong answers, the less useful the ensemble becomes. If they are 100% correlated with each other, the ensemble will give the exact same results as the individual classifiers. An ensemble doesn't *have to* improve results.

To put it in human terms, the "wisdom of the crowd" comes from people in the crowd having wrong beliefs about uncorrelated things. If everbody's wrong in the same way, there's no way to overcome that with volume. It could even be the opposite. Let's say 60% of people are wrong about a particular question. In a sample of 3 people, there's a decent chance that 2/3 will be among the 40% who are right. In a sample of a million people, the wrong belief is going to prevail.

To show the impact of correlation, I'll make one fake classifier, then a bunch of copies of it with 10% of the bits changed. Each fake classifier will have an accuracy of around 60%, but the errors will be highly correlated with each other.

In [ ]:
new_classifier = create_classifiers(ground_truth, 1, flip_ratio=.38)

def make_corr_classifiers(base_sys, flip=.1, num_classifiers=99):
    corr_classifiers = []
    for x in range(num_classifiers):
        new_sys = base_sys.copy()
        for count, bit in enumerate(base_sys):
            if rng.random() < flip:
                new_sys[count] = not(bit)
        corr_classifiers.append(new_sys)
    return corr_classifiers

corr_classifiers = make_corr_classifiers(new_classifier[0], flip=.1)


Previously, each system in the ensemble agreed with each other 52% of the time. Now it is around 82%.

In [34]:
sum(pd.Series(corr_classifiers[0]) == pd.Series(corr_classifiers[1])) / len(corr_classifiers[1])

0.81968

In [35]:
sum(pd.Series(corr_classifiers[1]) == pd.Series(corr_classifiers[2])) / len(corr_classifiers[1])

0.81992

each one agrees with the ground truth around 60% of the time.

In [36]:
print(f1_score(ground_truth, corr_classifiers[0]))
print(f1_score(ground_truth, corr_classifiers[1]))
print(f1_score(ground_truth, corr_classifiers[2]))

0.5974459410847756
0.5971458837167469
0.5971661537231704


If they were uncorrelated with each other, we saw that we could just keep adding more and more classifiers to the ensemble to reach any level of accuracy desired.

However, when they are highly correlated, there is a hard limit on how much the ensemble can improve things. 

Even with 99 classifiers in the ensemble, it only achieves an f1 score of .62. That's only a slight bump from the .60 achieved individually.

That was pretty surprising to me, even though I've seen similarly disappointing results from ensembles in the real world.

In [37]:
corr_maj = do_majority(corr_classifiers[:5])
print(classification_report(ground_truth, corr_maj, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.62      0.62      0.62     49918
           1       0.62      0.62      0.62     50082

    accuracy                           0.62    100000
   macro avg       0.62      0.62      0.62    100000
weighted avg       0.62      0.62      0.62    100000



In [38]:
corr_maj = do_majority(corr_classifiers[:99])
print(classification_report(ground_truth, corr_maj, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.62      0.62      0.62     49918
           1       0.62      0.62      0.62     50082

    accuracy                           0.62    100000
   macro avg       0.62      0.62      0.62    100000
weighted avg       0.62      0.62      0.62    100000



## How much does the ensemble help?

I've seen the recommendation that for real world data, an ensemble should contain about as many classifiers as categories. We're doing a binary classification here, so I'm going to use a size of 3 (ensuring there is a clear majority.)

The more accurate each classifier is, the more the ensemble improves things. While the ensemble of 60% accurate classifiers leads to 65% accuracy as a suite, an ensemble of 70% accurate classifiers will lead to 78% accuracy overall, and an ensemble of 80% classifiers leads to a 90% accuracy overall. It's a non-linear relationship.

In [17]:
def test_majority(flip_ratio):

    # using 3 components/classifiers in the ensemble for each test.
    classifiers = create_classifiers(ground_truth, 3, flip_ratio)

    ensemble_results = do_majority(classifiers)

    print(f"===Accuracy: {1- flip_ratio}")
    print(classification_report(ground_truth, ensemble_results, target_names=['0', '1']))

[test_majority(x) for x in 1 - np.array([.55, .6, .7, .8])]

===Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.57      0.57      0.57     49918
           1       0.57      0.57      0.57     50082

    accuracy                           0.57    100000
   macro avg       0.57      0.57      0.57    100000
weighted avg       0.57      0.57      0.57    100000

===Accuracy: 0.6
              precision    recall  f1-score   support

           0       0.65      0.65      0.65     49918
           1       0.65      0.65      0.65     50082

    accuracy                           0.65    100000
   macro avg       0.65      0.65      0.65    100000
weighted avg       0.65      0.65      0.65    100000

===Accuracy: 0.7
              precision    recall  f1-score   support

           0       0.78      0.78      0.78     49918
           1       0.78      0.78      0.78     50082

    accuracy                           0.78    100000
   macro avg       0.78      0.78      0.78    100000
weighted avg       0.7

[None, None, None, None]

## Soft Voting

Binary classifiers usually return a floating point number between 0 and 1, which gets rounded to a boolean classification. They will pretty much never return an exact 0 or 1, because they are based on probability distributions with infinite range -- to such models, nothing is impossible, just extremely unlikely.

Instead of rounding the individual classifiers, then going with the majority opinion (hard voting), we could combine the raw values from each classifier, then round that (soft voting).

We might think that because these classifiers are more precise, they're also more accurate, or that we will get a more accurate result by adding the floating point values together, and then taking the average. That's possible, but it could all be noise.

Let's see what that looks like. 

To model this, I will take the ground truth, flip `x%` of the bits, then add some noise to each one, but not enough to change what it will round to. If the bit is 1, I will select from a random uniform `[.5,1]` and if it's zero, `[0,.5]`.

In [18]:
def fuzzy_ground(input):
    """
    this takes the input and adds some random noise to the values
    so they are a float between 0 and 1 rather than a boolean. These fuzzy values 
    will round to the original booleans.
    """
    out = []
    for bit in input:
        if bit:
            # if 1, pick a number between .5 and 1 that will round to 1
            fuzzy = (rng.random() + 1) / 2
        else:
            # if 0, pick a number between 0 and .5 that will round to 0
            fuzzy = rng.random() / 2
        out.append(fuzzy)
    return out

def create_fuzzy_classifiers(ground_truth, num_classifiers=5, flip_ratio=.4):
    """
    generate classifiers, as before, then add some noise to each bit.
    """
    regular_classifiers = create_classifiers(ground_truth, num_classifiers, flip_ratio)

    fuzzy_classifiers = []
    for classifier in regular_classifiers:
        fuzzed = fuzzy_ground(classifier)
        fuzzy_classifiers.append(pd.Series(fuzzed))

    return fuzzy_classifiers

def fuzzy_ensemble(classifiers):
    rounded = np.round(np.mean(classifiers, axis=0))
    return rounded

To hammer the point here, I'm going to create an ensemble of 999 of these fuzzy systems. Looking at one of them, we can see that it still gives results that are correct 60% of the time. So, no funny business.

In [19]:
fuzzy_sys = create_fuzzy_classifiers(ground_truth, 999, .4)

# test that the first of the 999 classifiers has 60% f1 score
print(classification_report(ground_truth, round(fuzzy_sys[0]), target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.60      0.60      0.60     49918
           1       0.60      0.60      0.60     50082

    accuracy                           0.60    100000
   macro avg       0.60      0.60      0.60    100000
weighted avg       0.60      0.60      0.60    100000



As with the first example, because the errors are uncorrelated, we can keep adding more classifiers to the ensemble to get arbitrarily high accuracy.

In [20]:
fuzz_ensemble_results = fuzzy_ensemble(fuzzy_sys[:99])

print(classification_report(ground_truth, fuzz_ensemble_results, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     49918
           1       0.96      0.96      0.96     50082

    accuracy                           0.96    100000
   macro avg       0.96      0.96      0.96    100000
weighted avg       0.96      0.96      0.96    100000



## Soft voting can hurt ensemble performance

I'll look at 3 classifiers in the ensemble.

As with the hard voting experiment, the higher the base accuracy, the more benefit the ensemble gives. But the ensemble gives less of a benefit with soft voting than hard voting.


In [21]:
def test_fuzzy(flip_ratio):

    fuzzy_sys = create_fuzzy_classifiers(ground_truth, 3, flip_ratio)

    fuzz_ensemble_results = fuzzy_ensemble(fuzzy_sys)

    print(f"===Base Accuracy: {1- flip_ratio}")
    print(classification_report(ground_truth, fuzz_ensemble_results, target_names=['0', '1']))

In [22]:
[test_fuzzy(x) for x in 1 - np.array([.55, .6, .7, .8])]

===Base Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.56      0.57      0.56     49918
           1       0.57      0.56      0.56     50082

    accuracy                           0.56    100000
   macro avg       0.56      0.56      0.56    100000
weighted avg       0.56      0.56      0.56    100000

===Base Accuracy: 0.6
              precision    recall  f1-score   support

           0       0.62      0.63      0.62     49918
           1       0.62      0.62      0.62     50082

    accuracy                           0.62    100000
   macro avg       0.62      0.62      0.62    100000
weighted avg       0.62      0.62      0.62    100000

===Base Accuracy: 0.7
              precision    recall  f1-score   support

           0       0.74      0.74      0.74     49918
           1       0.74      0.74      0.74     50082

    accuracy                           0.74    100000
   macro avg       0.74      0.74      0.74    100000
weighte

[None, None, None, None]

The fuzz I added was from a random uniform distribution. I think a different way of modelling the noise would lead to a smaller gap between the soft and hard voting results. 

Soft and hard voting shouldn't be treated as interchangeable, and it's worth testing both when creating an ensemble. It's going to come down to whether the part being rounded off is signal or noise. if a value of `.4` really does indicate 4x more uncertainty about the precition than a value of `.1`, then there can be a benefit to soft voting. But if they're both just noise, the ensemble will perform better with hard voting.